In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



### prev_application 데이터 세트 기반의 EDA와 Feature Engineering 수행 후 학습 모델 생성/평가

### 라이브러리 및 데이터 세트 로딩

In [ ]:
import numpy as np
import pandas as pd
import gc
import time

%matplotlib inline

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [ ]:
#화면 크기 설정

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [ ]:
#application_train 데이터 feature 확인
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
print('Training data shape: ', app_train.shape) 
app_train.head() 

### 새로운 FEATURE 만들어서 APPLICATION_TRAIN 과 APPLICATION_TEST에 추가

In [ ]:
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')

In [ ]:
# app_train 과 app_test를 함께 가공을 해야하기 때문에 concat 이용하여 통합

apps = pd.concat([app_train, app_test])

In [ ]:
print('Size of application_train data', app_train.shape)
print('Size of application_test data', app_test.shape)
print('Size of apps', apps.shape)

In [ ]:
def get_apps_processed(apps):
    
    # EXT_SOURCE_X FEATURE 가공
    apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())
    
    # AMT_CREDIT 비율로 Feature 가공
    apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
    apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
    
    # AMT_INCOME_TOTAL 비율로 대출 금액 관련 Feature 가공
    apps['APPS_ANNUITY_INCOME_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CREDIT_INCOME_RATIO'] = apps['AMT_CREDIT']/apps['AMT_INCOME_TOTAL']
    apps['APPS_GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CNT_FAM_INCOME_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['CNT_FAM_MEMBERS']
    
    # DAYS_BIRTH, DAYS_EMPLOYED 비율로 소득/자산 관련 Feature 가공
    apps['APPS_EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED']/apps['DAYS_BIRTH']
    apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']
    apps['APPS_INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_BIRTH']
    apps['APPS_CAR_BIRTH_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_BIRTH']
    apps['APPS_CAR_EMPLOYED_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_EMPLOYED']
    
    return apps

### PREVIOUS_APPLICATION 로딩

In [ ]:
prev_app = pd.read_csv('../input/home-credit-default-risk/previous_application.csv')
print(prev_app.shape, apps.shape)

In [ ]:
prev_app.head()

### prev_app 컬럼과 Null 값 확인

In [ ]:
# Null 값을 확인하고 전체 데이터에서 비율을 보여주는 함수

def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
prev_app.info()

In [ ]:
missing_data(prev_app).head(20)

### SK_ID_CURR당 평균 SK_ID_PREV 건수 확인
- groupby로 평균 건수 구함
- boxplot으로 시각화

In [ ]:
# 시각화

sns.boxplot(prev_app.groupby('SK_ID_CURR')['SK_ID_CURR'].count())

### TARGET 유형에 따라 숫자형 피처들의 Histogram을 통해 확인
- application_train의 TARGET 값을 가져오기 위해 prev_app와 app_train을 조인 후 TARGET 유형에 따라 비교
- 숫자형 컬럼명 필터링

In [ ]:
# prev_app와 app_train을 inner 조인 ==> TARGET 값은 application_train(app_train) 데이터에만 있음
# ==> prev_app안에 TARGET 유형에 따라 FEATURE 확인을 하려면 조인할 필요가 있음







# app_train[['SK_ID_CURR', 'TARGET']] 두가지 컬럼만 갖고 옴
# on='SK_ID_CURR' 컬럼 기준으로 조인

app_prev_target = prev_app.merge(app_train[['SK_ID_CURR', 'TARGET']], on='SK_ID_CURR', how='left')
app_prev_target.shape

In [ ]:
# 지난 시간에 사용한 함수 그대로 사용
# TARGET 1 일때 값을 확인
# TARGET 0 일때 값을 확인

def show_hist_by_target(df, columns)
    cond_1 = (df['TARGET'] == 1)
    cond_0 = (df['TARGET'] == 0)
    
    for column in columns:
        fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 4), squeeze=False)
        sns.violinplot(x='TARGET', y=column, data=df, ax=axs[0][0] )
        sns.distplot(df[cond_0][column], ax=axs[0][1], label='0', color='blue')
        sns.distplot(df[cond_1][column], ax=axs[0][1], label='1', color='red')   

In [ ]:
app_prev_target.TARGET.value_counts()#(normalize=True)

### TARGET 유형에 따라 Category 피처들의 Histogram을 비교

In [ ]:
object_columns = app_prev_target.dtypes[app_prev_target.dtypes=='object'].index.tolist()
object_columns

In [ ]:
# category type 을 위한 시각화 함수

def show_category_by_target(df, columns):
    for column in columns:
        chart = sns.catplot(x=column, col="TARGET", data=df, kind="count")
        chart.set_xticklabels(rotation=65)

- NAME_CONTRACT_TYPE은 TARGET=1일때 CASH_LOAN의 비중이 약간 높음
- NAME_CONTRACT_STATUS(대출허가상태)는 TARGET=1일때 상대적으로 TARGET=0 대비 (당연히) Refused의 비율이 높음
- NAME_PAYMENT_TYPE(대출납부방법)는 TARGET=1일때 상대적으로 TARGET=0 대비 XNA의 비율이 약간 높음

In [ ]:
pd.crosstab(app_prev_target.TARGET, app_prev_target.NAME_CONTRACT_TYPE, dropna=False, normalize='all')

In [ ]:
pd.crosstab(app_prev_target.TARGET, app_prev_target.NAME_CONTRACT_STATUS, dropna=False, normalize='all')

In [ ]:
pd.crosstab(app_prev_target.NAME_CONTRACT_TYPE, app_prev_target.NAME_CONTRACT_STATUS, dropna=False)

In [ ]:
# 'AMT_ANNUITY' : 월 대출 지급액
app_prev_target.groupby(['NAME_CONTRACT_TYPE', 'NAME_GOODS_CATEGORY']).agg({'AMT_ANNUITY': ['mean', 'median', 'count', 'max']})

In [ ]:
# groupby 이용하여 더 많은 feature 추출할 수 있음
# 예) 직업에 따른 월급...교육과 직업...대부분 대출은 누가 받을까요...같은 feature 추출할 수 있음

app_train.groupby(['NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE']).agg({'AMT_INCOME_TOTAL': ['mean', 'median', 'count', 'max']})

# app_prev 데이터 세트 feature engineering 수행
- pandas groupby 단일 aggregation 함수 사용  
    - groupby SK_ID_CURR
    - SK_ID_CURR별 건수, AMT_CREDIT에 대한 평균, 최대 값

In [ ]:
# groupby SK_ID_CURR

prev_app.groupby('SK_ID_CURR')

In [ ]:
# 변수로 받아서 prev_group에 저장
prev_group = prev_app.groupby('SK_ID_CURR') # groupby 대상 컬럼명
prev_group.head()

In [ ]:
prev_agg = pd.DataFrame()
prev_agg['CNT'] = prev_group['SK_ID_CURR'].count()
prev_agg.head()

In [ ]:
prev_agg['AVG_CREDIT'] = prev_group['AMT_CREDIT'].mean()
prev_agg['MAX_CREDIT'] = prev_group['AMT_CREDIT'].max()
prev_agg['MIN_CREDIT'] = prev_group['AMT_CREDIT'].min()

prev_agg.head()

In [ ]:
# groupby agg()함수를 이용하여 여러개의 aggregation 함수 적용

prev_group = prev_app.groupby('SK_ID_CURR')
prev_agg1 = prev_group['AMT_CREDIT'].agg(['mean', 'max', 'min'])
prev_agg2 = prev_group['AMT_ANNUITY'].agg(['mean', 'max', 'min'])

prev_agg = prev_agg1.merge(prev_agg2, on='SK_ID_CURR', how='inner')
prev_agg.head()

### agg()에 dictionary를 이용하여 groupby 적용

In [ ]:
prev_amt_agg.columns = ['PREV_' + ('_').join(column).upper() for column in prev_amt_agg.columns.ravel()]
prev_amt_agg.head()

### prev 피처 가공 / 대출 신청액 대비 다른 금액 차이 및 비율 생성

In [ ]:
prev_app['PREV_CREDIT_DIFF'] = prev_app['AMT_APPLICATION'] - prev_app['AMT_CREDIT']
prev_app['PREV_GOODS_DIFF'] = prev_app['AMT_APPLICATION'] - prev_app['AMT_GOODS_PRICE']
prev_app['PREV_CREDIT_APPL_RATIO'] = prev_app['AMT_CREDIT']/prev_app['AMT_APPLICATION']
prev_app['PREV_ANNUITY_APPL_RATIO'] = prev_app['AMT_ANNUITY']/prev_app['AMT_APPLICATION']
prev_app['PREV_GOODS_APPL_RATIO'] = prev_app['AMT_GOODS_PRICE']/prev_app['AMT_APPLICATION']

### DAYS_XXX 피처의 365243 을 NULL로 변환하고, 첫번째 만기일과 마지막 만기일까지의 기간 가공

In [ ]:
prev_app['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
prev_app['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
# 첫번째 만기일과 마지막 만기일까지의 기간
prev_app['PREV_DAYS_LAST_DUE_DIFF'] = prev_app['DAYS_LAST_DUE_1ST_VERSION'] - prev_app['DAYS_LAST_DUE']

#### 기존 이자율 관련 컬럼이 null이 많아서 새롭게 간단한 이자율을 대출 금액과 대출 금액 납부 횟수를 기반으로 계산
- CNT_PAYMENT는 전체 납부해야하는 횟수, AMT_ANNUITY 월 별 납부해야하는 금액 -> 이것으로 간단하게 이자율 계산

In [ ]:
prev_app.info()

In [ ]:
all_pay = prev_app['AMT_ANNUITY'] * prev_app['CNT_PAYMENT']
prev_app['PREV_INTERESTS_RATE'] = (all_pay/prev_app['AMT_CREDIT'] - 1)/prev_app['CNT_PAYMENT']

In [ ]:
prev_app.iloc[:, -7:].head(10)

### SK_ID_CURR별로 NAME_CONTRACT_STATUS가 Refused 일 경우의 건수 및 과거 대출건 대비 비율
- Group by 기준 컬럼 기반에서 다른 컬럼들의 기준에 따라 세분화된 aggregation 수행

In [ ]:
prev_app['NAME_CONTRACT_STATUS'].value_counts()

In [ ]:
# SK_ID_CURR레벨로 groupby 된 count와 name_contract_status가 Refused일 때의 count
# NAME_CONTRACT_STATUS == 'Refused' 세부 기준으로 filtering 및 filtering 된 DataFrame에 groupby 적용

cond_refused = (prev_app['NAME_CONTRACT_STATUS'] == 'Refused')
prev_refused = prev_app[cond_refused]
prev_refused.shape, prev_app.shape

### Numerical Feature 시각화
- 함수 사용
- seaborn 사용

In [ ]:
f, ax = plt.subplots(2,3,figsize=(13, 10))


# DAYS_CREDIT: 현재 대출 신청 일 기준 과거 대출 신청 지난 기간 분포 확인
sns.distplot(app_bureau.DAYS_CREDIT.dropna(), kde=True, color="g", 
             ax=ax[0,0]).set_title('DAYS CREDIT Distribution')

# CREDIT_DAY_OVERDUE: 대출 신청 시 CB 크레딧 연체 일수
sns.distplot(app_bureau.CREDIT_DAY_OVERDUE.dropna(), kde=True, color="b",
             ax=ax[0,1]).set_title(' CREDIT DAY OVERDUE Distribution')


# DAYS_CREDIT_UPDATE: 대출 신청전 마지막 정보 받은 기간
sns.distplot(bureau.DAYS_CREDIT_UPDATE.dropna(), kde=True, 
             color="r", ax=ax[0,2]).set_title('DAYS CREDIT UPDATE Distribution')


# AMT_CREDIT_SUM_LIMIT: 신용 카드 현재 신용한도
sns.distplot(bureau.AMT_CREDIT_SUM_LIMIT.dropna(), kde=True, color="g", 
             ax=ax[1,0]).set_title(' Distribution')


# AMT_CREDIT_SUM_DEBT: 현재 채무 금액 총액
sns.distplot(bureau.AMT_CREDIT_SUM_DEBT.dropna(), kde=True, color="b",
             ax=ax[1,1]).set_title(' Distribution')

 
# AMT_CREDIT_SUM_OVERDUE: 최대 연체금액
sns.distplot(bureau.AMT_CREDIT_SUM_OVERDUE.dropna(), kde=True, 
             color="r", ax=ax[1,2]).set_title('DAYS CREDIT UPDATE Distribution')

In [ ]:
# 신용 한도를 초과하여 연체 된 고객이 있을까요?

sns.jointplot(x="AMT_CREDIT_SUM_LIMIT", y="AMT_CREDIT_SUM_OVERDUE", data=app_bureau)
plt.show()

# 연체 된 신용 한도를 초과하는 고객이 없음을 알 수 있다